# Import relevant modules and functions

In [1]:
# Basic modules and functions
import os
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [2]:
# Modules and functions for SQL handling
from sqlalchemy import text, create_engine, inspect

In [3]:
# Modules and functions to handle/derive geo information
#!pip install folium
#!pip install geopy

import folium
from folium import plugins
from folium.plugins import HeatMap
import geopy
import json
import pycountry

# Define functions to format geo information

## Get latitude and longitude data from address

In [4]:
# Define user agent
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.4 Safari/605.1.15'

In [5]:
# Initialize geocoder
geocoder = geopy.Nominatim(user_agent=USER_AGENT)

In [6]:
# Test geocoder
geocoder.geocode('Berlin, Germany')

Location(Berlin, 10117, Deutschland, (52.5170365, 13.3888599, 0.0))

In [7]:
# Use geocoder to derive coordinates, use memorization with dict

try: 
    city_dict.keys()
except:
    city_dict = dict()
    
def get_coordinates(df_cities_countries):
    """Get longitute and latitude based on city, country input"""
    coordinates = [] 
    for row in df_cities_countries.iterrows():
        city = row[1][0]
        country = row[1][1]

        lat, lng = city_dict.get(city, (999, 999))

        if lat  == 999:
            try:
                full_address = geocoder.geocode(f'{city}, {country}')
                if full_address == None:
                    full_address = geocoder.geocode(f'{country}')
            except:
                full_address = geocoder.geocode(f'{country}')

            lat = full_address.latitude
            lng = full_address.longitude
            coordinates.append([lat, lng])

            city_dict[city] = (lat, lng)

        else:
            coordinates.append([lat, lng])
    return coordinates

## Get country code for countries

In [8]:
# Example: Get country_codes with pycountry.countries().alpha_2
pycountry.countries.get(name="Germany").alpha_2

'DE'

In [9]:
# Get country codes for all countries in order table, using a dictionary for unknown countries and memorization

try: 
    country_dict.keys()
except:
    country_dict = {
        'USA' : 'US',
        'Venezuela' : 'VE',
        'UK' : 'GB'
    }

def get_country_codes(df_countries):
    country_codes=[]
    
    for country in df_countries:
        country_code = country_dict.get(country, 999)

        if country_code == 999:
            try: 
                country_code = pycountry.countries.get(name=country).alpha_2
            except:
                country_code = np.nan 
                print(country)
            country_dict[country] = country_code

        country_codes.append(country_code)
    return country_codes

# Format data and update database

_Tables containing (relevant) geo information: orders, customers, suppliers_

## Connect with postgresSQL database on AWS

In [10]:
from credentials import password #password is private

HOST = 'fenugreeks.chjdoa4nxtjk.eu-central-1.rds.amazonaws.com'
USERNAME = 'postgres'
PORT = '5432'
DB = 'northwind'
PASSWORD = password

In [11]:
#conn_string - for MAC local machine no username and password not needed -> f'postgresql://{HOST}:{PORT}/{DB}'
conn_string = f'postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DB}'

In [12]:
engine = create_engine(conn_string)

## Orders table

### Import data

In [13]:
orders = pd.read_sql_table('orders', engine)

In [14]:
orders.shape

(830, 17)

In [15]:
orders.head(3)

,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country,ship_latitude,ship_longitude,country_code
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,49.257789,4.031926,FR
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany,51.962510,7.625188,DE
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,Rua do Paço 67,Rio de Janeiro,RJ,05454-876,Brazil,-22.911014,-43.209373,BR


### Add geo information

In [16]:
# Get latitude and longitude
coordinates = get_coordinates(orders[['ship_city', 'ship_country']])
orders[['ship_latitude', 'ship_longitude']] = coordinates

In [17]:
# Get country codes
country_codes = get_country_codes(orders['ship_country'])
orders['country_code'] = country_codes

In [18]:
# Inspect update df
orders.head()

,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country,ship_latitude,ship_longitude,country_code
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France,49.257789,4.031926,FR
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany,51.962510,7.625188,DE
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,Rua do Paço 67,Rio de Janeiro,RJ,05454-876,Brazil,-22.911014,-43.209373,BR
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,2 rue du Commerce,Lyon,None,69004,France,45.757814,4.832011,FR
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,Boulevard Tirou 255,Charleroi,None,B-6000,Belgium,50.412033,4.443624,BE


In [22]:
# Explore results
i=2
city = folium.Map(location=coordinates[i], zoom_start=9)
folium.Marker(location=coordinates[i]).add_to(city)
city

### Update database on AWS

In [23]:
orders.to_sql('orders', engine, if_exists='replace', index=False)

In [24]:
engine.execute('ALTER TABLE orders ADD PRIMARY KEY ("order_id")')

## Customers table

### Import data

In [25]:
customers = pd.read_sql_table('customers', engine)

In [26]:
customers.shape

(91, 14)

In [27]:
customers.head(3)

,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax,latitude,longitude,country_code
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NULL,12209,Germany,030-0074321,030-0076545,52.517037,13.38886,DE
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,NULL,05021,Mexico,(5) 555-4729,(5) 555-3745,23.658512,-102.00771,MX
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,NULL,05023,Mexico,(5) 555-3932,NULL,23.658512,-102.00771,MX


### Add geo information

In [28]:
# Get latitude and longitude
coordinates = get_coordinates(customers[['city', 'country']])
customers[['latitude', 'longitude']] = coordinates

In [29]:
# Get country codes
country_codes = get_country_codes(customers['country'])
customers['country_code'] = country_codes

In [30]:
# Inspect update df
customers.head()

,customer_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax,latitude,longitude,country_code
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NULL,12209,Germany,030-0074321,030-0076545,52.517037,13.388860,DE
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,NULL,05021,Mexico,(5) 555-4729,(5) 555-3745,23.658512,-102.007710,MX
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,NULL,05023,Mexico,(5) 555-3932,NULL,23.658512,-102.007710,MX
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,NULL,WA1 1DP,UK,(171) 555-7788,(171) 555-6750,51.507322,-0.127647,GB
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,NULL,S-958 22,Sweden,0921-12 34 65,0921-12 34 67,65.583119,22.145954,SE


In [31]:
# Explore results
i=0
city = folium.Map(location=coordinates[i], zoom_start=9)
folium.Marker(location=coordinates[i]).add_to(city)
city

### Update database on AWS

In [32]:
customers.to_sql('customers', engine, if_exists='replace', index=False)

In [33]:
engine.execute('ALTER TABLE customers ADD PRIMARY KEY ("customer_id")')

## Suppliers table

### Import data

In [34]:
suppliers = pd.read_sql_table('suppliers', engine)

In [35]:
suppliers.shape

(29, 15)

In [36]:
suppliers.head(3)

,supplier_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax,homepage,latitude,longitude,country_code
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,NULL,EC1 4SD,UK,(171) 555-2222,NULL,NULL,51.507322,-0.127647,GB
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,NULL,#CAJUN.HTM#,29.975998,-90.078213,US
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,USA,(313) 555-5735,(313) 555-3349,NULL,42.268157,-83.731229,US


### Add geo information

In [37]:
# Get latitude and longitude
coordinates = get_coordinates(suppliers[['city', 'country']])
suppliers[['latitude', 'longitude']] = coordinates

In [38]:
# Get country codes
country_codes = get_country_codes(suppliers['country'])
suppliers['country_code'] = country_codes

In [39]:
# Inspect update df
suppliers.head()

,supplier_id,company_name,contact_name,contact_title,address,city,region,postal_code,country,phone,fax,homepage,latitude,longitude,country_code
0,1,Exotic Liquids,Charlotte Cooper,Purchasing Manager,49 Gilbert St.,London,NULL,EC1 4SD,UK,(171) 555-2222,NULL,NULL,51.507322,-0.127647,GB
1,2,New Orleans Cajun Delights,Shelley Burke,Order Administrator,P.O. Box 78934,New Orleans,LA,70117,USA,(100) 555-4822,NULL,#CAJUN.HTM#,29.975998,-90.078213,US
2,3,Grandma Kelly's Homestead,Regina Murphy,Sales Representative,707 Oxford Rd.,Ann Arbor,MI,48104,USA,(313) 555-5735,(313) 555-3349,NULL,42.268157,-83.731229,US
3,4,Tokyo Traders,Yoshi Nagase,Marketing Manager,9-8 Sekimai Musashino-shi,Tokyo,NULL,100,Japan,(03) 3555-5011,NULL,NULL,35.682839,139.759455,JP
4,5,Cooperativa de Quesos 'Las Cabras',Antonio del Valle Saavedra,Export Administrator,Calle del Rosal 4,Oviedo,Asturias,33007,Spain,(98) 598 76 54,NULL,NULL,43.360422,-5.845053,ES


In [40]:
# Explore results
i=0
city = folium.Map(location=coordinates[i], zoom_start=9)
folium.Marker(location=coordinates[i]).add_to(city)
city

### Update database on AWS

In [41]:
suppliers.to_sql('suppliers', engine, if_exists='replace', index=False)

In [42]:
engine.execute('ALTER TABLE suppliers ADD PRIMARY KEY ("supplier_id")')